In [6]:
import collections

In [7]:
class Node:
    
    def __init__(self,key,val,left=None,right=None,parent=None):
        self.key = key
        self.val = val
        self.left = left
        self.right = right
        self.parent = parent
        
    def replaceNodeData(self,key,val,left,right,parent=None):
        self.key = key
        self.val = val
        self.left = left
        self.right = right
        self.parent = parent
        
    def hasLeftChild(self):
        return self.left

    def hasRightChild(self):
        return self.right

    def isLeftChild(self):
        return self.parent and self.parent.left == self

    def isRightChild(self):
        return self.parent and self.parent.right == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.right or self.left)

    def hasAnyChildren(self):
        return self.right or self.left

    def hasBothChildren(self):
        return self.right and self.left

    def findSuccessor(self):
        
        succ = None
        
        if self.hasRightChild():
            succ = self.right.findMin()
        
        elif self.hasLeftChild(): # has only left child
            succ = self.left
            
        # no child returns succ as None without changing
        return succ
    
    def findMin(self):
        current = self
        
        while current.hasLeftChild():
            current = current.left
        
        return current
    
    def _spliceOut(self): # this operation will be done only to a Node with min-key value,
                            # which means it cannot have leftChild
        if self.isLeaf():
            if self.isLeftChild():
                self.parent.left = None
            else:
                self.parent.right = None
        
        else: # has right but not left
            if self.isLeftChild():
                self.parent.left = self.right
                self.right.parent = self.parent
            else:          
                self.parent.right = self.right
                self.right.parent = self.parent                
            

In [8]:
class BinarySearchTree:
    def __init__(self):
        self.root = None
        self.size = 0
    
    def put(self,key,val):
        if not self.root:
            self.root = Node(key,val)
        
        else:
            self._put(key,val,self.root)
        
        self.size += 1
    
    def _put(self,key,val,currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                self._put(key,val,currentNode.left)
            else:
                currentNode.left = Node(key,val,parent=currentNode)
                
        else:
            if currentNode.hasRightChild():
                self._put(key,val,currentNode.right)
            else:
                currentNode.right = Node(key,val,parent=currentNode)
                
    def get(self,key):
        if not self.root:
            return None
        
        targetNode = self._get(key,self.root)
        
        if targetNode:
            return targetNode.val
        else:
            return None
        
    def _get(self,key,currentNode):
        if not currentNode:
            return None
        
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                return self._get(key,currentNode.left)
            else:
                return None
        
        elif key > currentNode.key:
            if currentNode.hasRightChild():
                return self._get(key,currentNode.right)
            else:
                return None
            
        else: # key == currentNode.key:
            return currentNode
    
    def delete(self, key):
        if self.size > 1:
            nodeToRemove = self._get(key, self.root)
            if not nodeToRemove:
                raise KeyError('Key is not in the tree.')
                
            self._remove(nodeToRemove)
            self.size -= 1
            
        elif self.size == 1:
            if self.root.key == key:
                self.root = None
                self.size =0
                
            else:
                raise KeyError('Key is not in the tree.')
        
        else: # seif.size == 0
            raise KeyError('Key is not in the tree (because empty tree is empty).')
    
    def _remove(self, targetNode):
        
        if targetNode.isLeaf():
            if targetNode.isLeftChild():
                targetNode.parent.left = None
            else:
                targetNode.parent.right = None
            
        elif targetNode.hasBothChildren():
            succ = targetNode.findSuccessor()
            succ._spliceOut()
            targetNode.key = succ.key
            targetNode.val = succ.val
        
        else: # targetNode has one child
            if targetNode.hasLeftChild():
                if targetNode.isLeftChild():
                    targetNode.parent.left = targetNode.left
                    targetNode.left.parent = targetNode.parent
                elif targetNode.isRightChild():
                    targetNode.parent.right = targetNode.left
                    targetNode.left.parent = targetNode.parent
                else: # targetNode is the root
                    targetNode.replaceNodeData(targetNode.left.key,
                                                targetNode.left.val,
                                                tagetNode.left.left,
                                                targetNode.left.right)
            else: # targetNode has rightChild
                if targetNode.isLeftChild():
                    targetNode.parent.left = targetNode.right
                    targetNode.right.parent = targetNode.parent
                elif targetNode.isRightChild():
                    targetNode.parent.right = targetNode.right
                    targetNode.right.parent = targetNode.parent
                else:
                    targetNode.replaceNodeData(targetNode.right.key,
                                               targetNode.right.val,
                                               targetNode.right.left,
                                               targetNode.right.right)
            
    def __getitem__(self,key):
        return self.get(key)
    
    def __setitem__(self,key,val):
        return self.put(key,val)
    
    def __len__(self):
        return self.size
    
    def __contains__(self,key):
        if self._get(key,self.root):
            return True
        else:
            return False
        
    def __delitem__(self,key):
        self.delete(key)
    

In [9]:
def levelOrderPrint(tree):
    
    if not tree.root:
        return
    nodes = collections.deque([tree.root])

    count_nodes_current_level = 1
    count_nodes_next_level = 0
    
    while count_nodes_current_level > 0:
        
        currentNode = nodes.popleft()
        count_nodes_current_level -= 1
        print(currentNode.key, end=' ')
        
        if currentNode.left:
            nodes.append(currentNode.left)
            count_nodes_next_level += 1
        if currentNode.right:
            nodes.append(currentNode.right)
            count_nodes_next_level += 1
        
        if count_nodes_current_level == 0:
            count_nodes_current_level, count_nodes_next_level = count_nodes_next_level, count_nodes_current_level
            print('\n')    

In [16]:
mytree = BinarySearchTree()
mytree.put(5,'five')
mytree.put(10,'ten')
mytree.put(2,'two')
mytree.put(8, 'eight')
mytree.put(7, 'seven')
mytree.put(9,'nine')
levelOrderPrint(mytree)

5 

2 10 

8 

7 9 



In [12]:
def trimBST(tree, minVal, maxVal):

    def trim(node,minVal,maxVal):
        if not node:
            return
        
        node.left = trim(node.left,minVal,maxVal)
        node.right = trim(node.right,minVal,maxVal)
        
        if node.key < minVal:
            return node.right
        
        if node.key > maxVal:
            return node.left
        
        else:
            return node
        
    if not tree.root:
        return None
    
    trim(tree.root,minVal,maxVal)

In [17]:
trimBST(mytree,3,9)

In [18]:
levelOrderPrint(mytree)

5 

8 

7 9 

